# 4 – Data Drift Analysis with H2O + Alibi-Detect

Este notebook:
- Carga el dataset original `insurance_company_original.csv`
- Separa la variable objetivo `CARAVAN`
- Genera un conjunto de validación
- Calcula métricas baseline del modelo H2O guardado en `../models/h2o_automl_model`
- Simula **data drift** sintético sobre las features
- Evalúa la caída de desempeño (accuracy, F1, AUC)
- Detecta drift en las distribuciones con **Alibi-Detect** (MMD + ChiSquare)
- Genera una gráfica comparativa de métricas.

*Ajusta las rutas si tu estructura de carpetas es distinta.*

In [ ]:
!pip install h2o alibi-detect scikit-learn matplotlib pandas numpy


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import h2o
from alibi_detect.cd import MMDDrift, ChiSquareDrift

# Estilo de gráficos
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['figure.dpi'] = 120


## 1. Carga de datos

In [ ]:
DATA_PATH = '../data/raw/insurance_company_original.csv'  # ajusta si es necesario
TARGET_COL = 'CARAVAN'

df = pd.read_csv(DATA_PATH, delimiter=';')
print(df.shape)
df.head()


## 2. Limpieza ligera y separación Train / Validation

In [ ]:
# Separar target
y = df[TARGET_COL]
X = df.drop(columns=[TARGET_COL])

# Forzamos a numérico donde se pueda (solo por robustez)
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='ignore')

# División train / validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_val.shape


## 3. Inicializar H2O y cargar el modelo

In [ ]:
h2o.init()

MODEL_PATH = '../models/h2o_automl_model'  # ajusta si es necesario

model = h2o.load_model(MODEL_PATH)
model


## 4. Métricas baseline sobre el conjunto de validación

In [ ]:
# Convertir a H2OFrame
hX_val = h2o.H2OFrame(X_val)

# Predicciones con el modelo H2O
pred_h2o = model.predict(hX_val).as_data_frame()

# Intentamos obtener probabilidades y clase predicha
y_pred = None
y_proba = None

if 'predict' in pred_h2o.columns:
    # Mapeamos a entero si viene como string/categoría
    try:
        y_pred = pred_h2o['predict'].astype(int).values
    except ValueError:
        y_pred = pred_h2o['predict'].map({'0': 0, '1': 1}).values

# Probabilidades (para AUC)
if 'p1' in pred_h2o.columns:
    y_proba = pred_h2o['p1'].values

# Métricas baseline
baseline_metrics = {}

baseline_metrics['accuracy'] = accuracy_score(y_val, y_pred)
baseline_metrics['f1'] = f1_score(y_val, y_pred)

if y_proba is not None and len(np.unique(y_val)) == 2:
    baseline_metrics['auc'] = roc_auc_score(y_val, y_proba)
else:
    baseline_metrics['auc'] = np.nan

baseline_metrics


## 5. Simulación de Data Drift sintético

In [ ]:
X_drift = X_val.copy()

# Columnas numéricas y categóricas
num_cols = X_drift.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_drift.select_dtypes(exclude=[np.number]).columns.tolist()

print(f'Numéricas: {len(num_cols)} columnas')
print(f'Categóricas: {len(cat_cols)} columnas')

# 5.1 Drift en numéricas: mean-shift + ruido
for col in num_cols:
    shift = np.random.normal(loc=0.3, scale=0.1)  # desplazar la media ~30%
    noise = np.random.normal(loc=0.0, scale=0.5, size=len(X_drift))
    X_drift[col] = X_drift[col] * (1 + shift) + noise

# 5.2 Drift categórico: concentrar en la moda
for col in cat_cols:
    mode_val = X_drift[col].mode()[0]
    mask = np.random.rand(len(X_drift)) < 0.7  # 70% hacia la categoría dominante
    X_drift.loc[mask, col] = mode_val

X_drift.head()


## 6. Métricas con Data Drift

In [ ]:
# Predicción sobre el dataset con drift
hX_drift = h2o.H2OFrame(X_drift)
pred_drift = model.predict(hX_drift).as_data_frame()

# Extraer predicciones
if 'predict' in pred_drift.columns:
    try:
        y_pred_drift = pred_drift['predict'].astype(int).values
    except ValueError:
        y_pred_drift = pred_drift['predict'].map({'0': 0, '1': 1}).values
else:
    raise RuntimeError("La predicción de H2O no tiene columna 'predict'. Revisa el modelo.")

# Probabilidades para AUC
if 'p1' in pred_drift.columns:
    y_proba_drift = pred_drift['p1'].values
else:
    y_proba_drift = None

drift_metrics = {}
drift_metrics['accuracy'] = accuracy_score(y_val, y_pred_drift)
drift_metrics['f1'] = f1_score(y_val, y_pred_drift)

if y_proba_drift is not None and len(np.unique(y_val)) == 2:
    drift_metrics['auc'] = roc_auc_score(y_val, y_proba_drift)
else:
    drift_metrics['auc'] = np.nan

drift_metrics


## 7. Caída de desempeño y alerta

In [ ]:
performance_drop = {}
alerts = {}

for m in baseline_metrics.keys():
    base = baseline_metrics[m]
    new = drift_metrics[m]
    if np.isnan(base) or np.isnan(new):
        performance_drop[m] = np.nan
        alerts[m] = False
        continue
    drop = (base - new) / base
    performance_drop[m] = drop
    alerts[m] = drop > 0.10  # umbral 10%

performance_drop, alerts


## 8. Detección de Data Drift con Alibi-Detect

In [ ]:
# 8.1 Drift en numéricas con MMDDrift
X_ref_num = X_val[num_cols].values.astype(np.float32)
X_test_num = X_drift[num_cols].values.astype(np.float32)

cd_num = MMDDrift(X_ref_num, p_val=0.05)
preds_num = cd_num.predict(X_test_num, return_p_val=True, return_distance=True)
preds_num


In [ ]:
# 8.2 Drift en categóricas con ChiSquareDrift (si hay categóricas)
if len(cat_cols) > 0:
    X_ref_cat = X_val[cat_cols].astype(str).to_numpy()
    X_test_cat = X_drift[cat_cols].astype(str).to_numpy()

    cd_cat = ChiSquareDrift(X_ref_cat, p_val=0.05)
    preds_cat = cd_cat.predict(X_test_cat, return_p_val=True)
    preds_cat
else:
    print("No hay columnas categóricas; se omite ChiSquareDrift.")


## 9. Gráfica comparativa de métricas

In [ ]:
metrics = list(baseline_metrics.keys())
baseline_vals = [baseline_metrics[m] for m in metrics]
drift_vals = [drift_metrics[m] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

plt.figure()
plt.bar(x - width/2, baseline_vals, width, label='Baseline')
plt.bar(x + width/2, drift_vals, width, label='Drift')

plt.xticks(x, metrics)
plt.ylim(0, 1.0)
plt.ylabel('Valor métrica')
plt.title('Baseline vs Drift – Desempeño del modelo')
plt.legend()
plt.show()


## 10. Resumen para el reporte

En este experimento:

"
"- Se tomó el dataset original `insurance_company_original.csv` y se separó la variable objetivo `CARAVAN`.
"
"- Se definió un conjunto de validación y se calcularon las métricas baseline del modelo H2O (`accuracy`, `F1` y `AUC`).
"
"- Se generó un conjunto de monitoreo con **data drift sintético**, aplicando desplazamientos en las distribuciones numéricas y concentrando las variables categóricas en sus valores más frecuentes.
"
"- Se evaluó nuevamente el desempeño del modelo sobre el dataset con drift y se midió la **caída relativa de desempeño**.
"
"- Se utilizaron detectores de drift de la librería **Alibi-Detect**: `MMDDrift` para datos numéricos y `ChiSquareDrift` para datos categóricos.
"
"- Cuando la caída de las métricas supera el umbral del 10%, se activa una **alerta de degradación de modelo**, lo cual sugiere revisar la calidad de los datos entrantes y considerar un retrenamiento.

"
"Este enfoque ilustra cómo integrar monitoreo de data drift y de desempeño del modelo como parte de una estrategia de MLOps para mantenimiento proactivo en producción."
